In [13]:
import tensorflow.keras as keras
import numpy as np
import importlib

import kaggle_functions as kaggle
import model10_wideresnet
importlib.reload(kaggle); 
importlib.reload(model10_wideresnet);
from model10_wideresnet import Model

model_number = 'model10'

### Create datasets

In [8]:
train_dataset, valid_dataset, valid_labels = kaggle.load_train_as_dataset()

batch_size = 32
train_dataset_augmented, epoch_length = kaggle.augment_dataset(train_dataset, batch_size)

In [ ]:
kaggle.show_images(train_dataset_augmented, 9)

### Train new model

In [14]:
Model().build(None).summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 96, 96, 1)]  0           []                               
                                                                                                  
 conv2d_10 (Conv2D)             (None, 48, 48, 16)   160         ['input_2[0][0]']                
                                                                                                  
 batch_normalization_7 (BatchNo  (None, 48, 48, 16)  64          ['conv2d_10[0][0]']              
 rmalization)                                                                                     
                                                                                                  
 re_lu_7 (ReLU)                 (None, 48, 48, 16)   0           ['batch_normalization_7[0][

In [15]:
# Fit model
# See {model}.py for specific training instructions, like epochs or valid_patience
model_name = 'WideResNet_1'

model, history = kaggle.train_model(
    Model().build(None), train_dataset_augmented, valid_dataset, 
    epochs=200, valid_patience=20, epoch_length=epoch_length)

Epoch 1/200
298/298 [==============================] - 167s 516ms/step - loss: 2.2697 - accuracy: 0.2489 - val_loss: 2.6237 - val_accuracy: 0.2233 - lr: 5.0000e-04
Epoch 2/200
298/298 [==============================] - 147s 494ms/step - loss: 2.0617 - accuracy: 0.3254 - val_loss: 2.3442 - val_accuracy: 0.3003 - lr: 5.0000e-04
Epoch 3/200
298/298 [==============================] - 147s 494ms/step - loss: 1.9363 - accuracy: 0.3607 - val_loss: 2.7396 - val_accuracy: 0.2410 - lr: 5.0000e-04
Epoch 4/200
298/298 [==============================] - 142s 478ms/step - loss: 1.8536 - accuracy: 0.3931 - val_loss: 2.9307 - val_accuracy: 0.2998 - lr: 5.0000e-04
Epoch 5/200
298/298 [==============================] - 118s 397ms/step - loss: 1.7722 - accuracy: 0.4265 - val_loss: 2.3492 - val_accuracy: 0.3095 - lr: 5.0000e-04
Epoch 6/200
298/298 [==============================] - 118s 397ms/step - loss: 1.7108 - accuracy: 0.4481 - val_loss: 3.8495 - val_accuracy: 0.2065 - lr: 5.0000e-04
Epoch 7/200
298/

In [ ]:
# Save model
model.save(f'models/{model_number}/{model_name}')

# Plot model statistics during training
kaggle.plot_model_history(history, [['accuracy', 'val_accuracy'], ['loss', 'val_loss']])

### Fine-tune

In [ ]:
# Fine-tune model
fine_model, history = kaggle.fine_tune_model_filepath(
    f'models/{model_number}/WideResNet_1',
    train_dataset.batch(32), valid_dataset, 
    epochs=3, learning_rate=0.00001)

In [11]:
# Save fine-tuned model
fine_model_name = 'WideResNet_2'
fine_model.save(f'models/{model_number}/{fine_model_name}')

INFO:tensorflow:Assets written to: models/model7/DeeperVGG_7\assets


### Evaluate

In [ ]:
model_name = ''
model = keras.models.load_model(f'models/{model_number}/{model_name}')
test_pred = np.argmax(model.predict(valid_dataset.batch(128)), axis=1)

kaggle.print_accuracy(valid_labels, test_pred)
kaggle.plot_confusion_matrix(valid_labels, test_pred)

### Generate test predictions

In [ ]:
model_name = ''
kaggle.generate_test_pred_filepath(f'models/{model_number}/{model_name}')

### Hypertune models

In [ ]:
tuner = kaggle.hypertune_model(
    Model(), train_dataset_augmented, valid_dataset, 
    model_number, 'hypertuner2021-11-30', epochs=200, trials=5)

In [ ]:
tuner = kaggle.load_hypertuner(Model(), model_number, '')